In [41]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Set the paths for the parent directory and the train/test directories
parent_dir = '/home/karthik/karthik/sem6/facerecog/dataset'
train_dir = os.path.join(parent_dir, 'train-cropped')
test_dir = os.path.join(parent_dir, 'test-cropped')

In [42]:
IMG_SIZE = 224
CHANNELS = 3

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Crop and resize the first face found
    if len(faces) > 0:
        x, y, w, h = faces[0]
        image = image[y:y+h, x:x+w]
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        # Convert the image to a numpy array and normalize its values
        image = np.array(image, dtype=np.float32)
        image /= 255.0

        return image
    else:
        return None


In [43]:
label_map = {
    'abhiram': 0,
    'jagadeeshan': 1,
    'jagannath': 2,
    'kiran': 3,
    'kodam': 4,
    'praneeth': 5,
    'rahul': 6,
    'samarth': 7,
    'sumanth': 8,
    'vinay': 9
}


In [44]:
def load_dataset(directory, label_map):
    images = []
    labels = []
    for label_name in os.listdir(directory):
        label = label_map[label_name]
        label_dir = os.path.join(directory, label_name)
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            image = preprocess_image(image_path)
            if image is not None:
                images.append(image)
                labels.append(label)
    images = np.stack(images)
    labels = np.array(labels, dtype=np.int32)
    return images, labels


In [45]:
train_images, train_labels = load_dataset(train_dir, label_map)
test_images, test_labels = load_dataset(test_dir, label_map)


In [46]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(os.listdir(train_dir)), activation='softmax')
])


In [47]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2%}')

# Predict the labels of the test images
predicted_labels = model.predict(test_images)

# Print the images and predicted labels for the test data
for i in range(len(test_images)):
    image = test_images[i]
    label = test_labels[i]
    predicted_label = np.argmax(predicted_labels[i])
    print(f'Image {i+1}: Actual label - {label}, Predicted label - {predicted_label}')


Epoch 1/10
4/4 [==============================] - 9s 2s/step - loss: 4.5248 - accuracy: 0.0777 - val_loss: 2.2697 - val_accuracy: 0.2222
Epoch 2/10
4/4 [==============================] - 7s 2s/step - loss: 2.2814 - accuracy: 0.2136 - val_loss: 2.3231 - val_accuracy: 0.0556
Epoch 3/10
4/4 [==============================] - 7s 2s/step - loss: 2.2308 - accuracy: 0.2330 - val_loss: 2.2770 - val_accuracy: 0.1667
Epoch 4/10
4/4 [==============================] - 7s 2s/step - loss: 2.1469 - accuracy: 0.2427 - val_loss: 2.1555 - val_accuracy: 0.3333
Epoch 5/10
4/4 [==============================] - 8s 2s/step - loss: 2.0283 - accuracy: 0.4175 - val_loss: 2.0481 - val_accuracy: 0.3333
Epoch 6/10
4/4 [==============================] - 8s 2s/step - loss: 1.8043 - accuracy: 0.4854 - val_loss: 1.8045 - val_accuracy: 0.3889
Epoch 7/10
4/4 [==============================] - 8s 2s/step - loss: 1.4241 - accuracy: 0.5340 - val_loss: 1.5541 - val_accuracy: 0.5556
Epoch 8/10
4/4 [=========================